<a href="https://colab.research.google.com/github/Collin-Campbell/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/module4-Deploy/LS_DS_424_Deploy_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Train Practice

## *Data Science Unit 4 Sprint 2 Module 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [1]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Ftrl
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

from io import BytesIO
import numpy as np
import requests

In [2]:
def load_quickdraw10(url):
    r = requests.get(url, stream = True)
    data = np.load(BytesIO(r.raw.read()))
    X = data['arr_0']
    y = data['arr_1']

    class_names = ['apple',
    'anvil',
    'airplane',
    'banana',
    'The Eiffel Tower',
    'The Mona Lisa',
    'The Great Wall of China',
    'alarm clock',
    'ant',
    'asparagus']

    # Need this line to randomly shuffle both the X & y at the same time.
    X, y = shuffle(X, y)

    # splitting training into 80% train and 20% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print('X_train shape = {}'.format(X_train.shape))
    print('y_train shape = {}'.format(y_train.shape))
    print('\n')
    print('X_test shape = {}'.format(X_test.shape))
    print('y_test shape = {}'.format(y_test.shape))
    print('\n \n')

    return X_train, X_test, y_train, y_test

In [3]:
url = 'https://github.com/skhabiri/ML-ANN/raw/main/data/quickdraw10.npz'
X_train, X_test, y_train, y_test = load_quickdraw10(url)

X_train shape = (80000, 784)
y_train shape = (80000,)


X_test shape = (20000, 784)
y_test shape = (20000,)

 



In [5]:
type(Adam), Adam

(abc.ABCMeta, tensorflow.python.keras.optimizer_v2.adam.Adam)

In [19]:
def create_model(path, lr=.001, opt_name=Adam):

    X_train, X_test, y_train, y_test = load_quickdraw10(path)

    # Define the keras model
    model = Sequential()
    model.add(Dense(588, input_dim=784, activation='sigmoid'))
    model.add(Dense(441, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    # Compile the keras model
    # opt_name is an abstract base class and Adam, SGD ,... are childeren of that super class
    # Hence all the childeren of that abc class are pointing to that abc parent class 
    # and can replace opt_name
    opt = opt_name(learning_rate=lr)
    print("Optimizer type:\n", type(opt_name), "\n", type(opt))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    model.summary()

    return model

In [16]:
stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [24]:
# To do this, add this to callback function:

cpoint_callback = ModelCheckpoint("weights_best.ah5", 
                         verbose=1, save_weights_only=True)

In [25]:
Adam_model = create_model(url, lr=.0001, opt_name=Adam)
Adam_model.fit(X_train, y_train,
                                epochs=10,
                                batch_size=64,
                                callbacks=[stop_callback, cpoint_callback],
                                validation_data=(X_test,y_test)
                                )

X_train shape = (80000, 784)
y_train shape = (80000,)


X_test shape = (20000, 784)
y_test shape = (20000,)

 

Optimizer type:
 <class 'abc.ABCMeta'> 
 <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 588)               461580    
_________________________________________________________________
dense_13 (Dense)             (None, 441)               259749    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                4420      
Total params: 725,749
Trainable params: 725,749
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.1869 - accuracy: 0.6280 - val_loss: 0.7602 - val_accuracy: 0.7712

Epoch 00001: saving mod

In [28]:
m = create_model(url)  # Start with same architecture
m.load_weights('weights_best.ah5')  # Load instead of train
m.summary()

X_train shape = (80000, 784)
y_train shape = (80000,)


X_test shape = (20000, 784)
y_test shape = (20000,)

 

Optimizer type:
 <class 'abc.ABCMeta'> 
 <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 588)               461580    
_________________________________________________________________
dense_16 (Dense)             (None, 441)               259749    
_________________________________________________________________
dense_17 (Dense)             (None, 10)                4420      
Total params: 725,749
Trainable params: 725,749
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dens

In [29]:
m.evaluate(X_test, y_test, verbose=1)

625/625 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.8367


[0.5258622765541077, 0.8409000039100647]

In [21]:
# To save entire model:

Adam_model.save('my_Adam_model')


# To load the model back in:

new_model = tf.keras.models.load_model('my_Adam_model')

INFO:tensorflow:Assets written to: my_Adam_model/assets


In [23]:
new_model.evaluate(X_test, y_test)

625/625 [==============================] - 1s 1ms/step - loss: 0.4709 - accuracy: 0.8635


[0.4709043502807617, 0.8634999990463257]

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 